In [ ]:
import math
import random
import operator 
import numpy as np
import pandas as pd
from scipy import stats

from deap import base    # core
from deap import creator # core
from deap import tools	 # core
from deap import gp 
from deap import algorithms


In [ ]:
csv_path = "C:\\Users\\roderickzzc\\Desktop\\Quant project\\data\\"
stockname="close"
close = pd.read_csv(csv_path + stockname + ".csv", index_col = 0, parse_dates = True)
stockname="open"
op = pd.read_csv(csv_path + stockname + ".csv", index_col = 0, parse_dates = True)
stockname="high"
high = pd.read_csv(csv_path + stockname + ".csv", index_col = 0, parse_dates = True)
stockname="low"
low = pd.read_csv(csv_path + stockname + ".csv", index_col = 0, parse_dates = True)
stockname="TURNOVER_VALUE"
turnover = pd.read_csv(csv_path + stockname + ".csv", index_col = 0, parse_dates = True)
stockname="vol"
vol = pd.read_csv(csv_path + stockname + ".csv", index_col = 0, parse_dates = True)

vol=vol.loc['2008-01-04':'2017-01-16']/1000000
turnover=turnover.loc['2008-01-04':'2017-01-16']/1000000
high=high.loc['2008-01-04':'2017-01-16']
low=low.loc['2008-01-04':'2017-01-16']
op=op.loc['2008-01-04':'2017-01-16']
close=close.loc['2008-01-04':'2017-01-16']

turnover=turnover[vol.columns]

In [ ]:
close=close.fillna(close.mean())
close=close.dropna(how='all',axis=1)
op=op.fillna(op.mean())
op=op.dropna(how='all',axis=1)
high=high.fillna(high.mean())
high=high.dropna(how='all',axis=1)
low=low.fillna(low.mean())
low=low.dropna(how='all',axis=1)
turnover=turnover.fillna(turnover.mean())
turnover=turnover.dropna(how='all',axis=1)
vol=vol.fillna(vol.mean())
vol=vol.dropna(how='all',axis=1)


In [ ]:
def _rank(data):
    return pd.DataFrame.rank(close,axis=1)

def _rolling_rank(data):
    value = stats.rankdata(data,axis=0)[-1]
    return value

def _cube(data):
    return np.square(data)*data

def _square(data):
    return np.square(data)

def _delta(data):
    delta=np.diff(data,axis=0)
    delta=np.insert(delta, 0, 0, axis=0)
    return pd.DataFrame(delta).set_index(exp_return.index).set_axis(close.columns, axis=1,inplace=False)

def _delay(data,n):
    period=n
    value = data.shift(period)
    value = np.nan_to_num(value)
    return pd.DataFrame(value).set_index(exp_return.index).set_axis(close.columns, axis=1,inplace=False)
        
def _corr(data1,data2,n):
    return data1.rolling(n).corr(data2)

def _cov(data1,data2,n):
    return data1.rolling(n).cov(data2)
    
def _ts_sum(data,n):
    data=data.rolling(n).sum()
    data=np.nan_to_num(data)
    return pd.DataFrame(data).set_index(exp_return.index).set_axis(close.columns, axis=1,inplace=False)
        
def _sma(data,n):
    data=data.rolling(n).mean()
    data=np.nan_to_num(data)
    return pd.DataFrame(data).set_index(exp_return.index).set_axis(close.columns, axis=1,inplace=False)

def _stddev(data,n): 
    data=data.rolling(n).std()
    data=np.nan_to_num(data)
    return pd.DataFrame(data).set_index(exp_return.index).set_axis(close.columns, axis=1,inplace=False)

def _ts_rank(data,n):
    data=data.rolling(n).apply(_rolling_rank)
    data=np.nan_to_num(data)
    return pd.DataFrame(data).set_index(exp_return.index).set_axis(close.columns, axis=1,inplace=False)
    
def _ts_argmin(data,n):
    data=data.rolling(n).apply(np.argmin)
    data=np.nan_to_num(data)
    return pd.DataFrame(data).set_index(exp_return.index).set_axis(close.columns, axis=1,inplace=False)

def _ts_argmax(data,n):
    data=data.rolling(n).apply(np.argmax)
    data=np.nan_to_num(data)
    return pd.DataFrame(data).set_index(exp_return.index).set_axis(close.columns, axis=1,inplace=False)
        
def _ts_min(data,n):
    data=data.rolling(n).min()
    data=np.nan_to_num(data)
    return pd.DataFrame(data).set_index(exp_return.index).set_axis(close.columns, axis=1,inplace=False)
        
def _ts_max(data,n):
    data=data.rolling(n).min()
    data=np.nan_to_num(data)
    return pd.DataFrame(data).set_index(exp_return.index).set_axis(close.columns, axis=1,inplace=False)
        
def _ts_argmaxmin(data,n):
    return _ts_argmax(data,n) - _ts_argmin(data,n)

def protectedDiv(data1,data2):
    df=data1/data2
    return df

def linear_weight(d):
    w = list([n/sum(range(1,d+1)) for n in range(1,d+1)])
    def g(x):
        return sum(w*x) 
    return g

def linear_decay(data,n):
    return data.rolling(window=n).apply(linear_weight(n))

def exponential_average(data):
    return data.ewm(com=0.5).mean()

def iden(data):
    return data

In [ ]:
datype=pd.core.frame.DataFrame

pset = gp.PrimitiveSetTyped("MAIN", [datype,datype,datype,datype,datype,datype], datype)  

pset.addPrimitive(operator.add, [datype,datype],datype) 
pset.addPrimitive(operator.sub, [datype,datype],datype)
pset.addPrimitive(operator.mul, [datype,datype],datype)
pset.addPrimitive(protectedDiv, [datype,datype],datype)
pset.addPrimitive(operator.neg, [datype],datype)

pset.addPrimitive(_cube, [datype],datype)
pset.addPrimitive(_square, [datype],datype)
pset.addPrimitive(_rank, [datype],datype)
pset.addPrimitive(_delta, [datype],datype)

pset.addPrimitive(_delay, [datype,int],datype)
pset.addPrimitive(_corr, [datype,datype,int],datype)
pset.addPrimitive(_cov, [datype,datype,int],datype)
pset.addPrimitive(_ts_sum, [datype,int],datype)
pset.addPrimitive(_sma, [datype,int],datype)
pset.addPrimitive(_stddev, [datype,int],datype)
pset.addPrimitive(_ts_rank, [datype,int],datype)
pset.addPrimitive(_ts_argmin, [datype,int],datype)
pset.addPrimitive(_ts_argmax, [datype,int],datype)
pset.addPrimitive(_ts_argmaxmin, [datype,int],datype)
pset.addPrimitive(linear_decay, [datype,int],datype)
pset.addPrimitive(iden, [int],int)
#pset.addPrimitive(math.cos, 1)
#pset.addPrimitive(math.sin, 1)


pset.addEphemeralConstant('int',lambda: random.randint(1, 20), int)


pset.renameArguments(ARG0='close',ARG1='op',ARG2='high',ARG3='low',ARG4='turnover',ARG5='vol')  



In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)


In [ ]:
# calculate the forward returns
forward_day = 5
stock = pd.read_csv(csv_path + "close" + ".csv", index_col = 0, parse_dates = True)
stockforward = stock.shift(periods = -forward_day, freq = None, axis = 0)
# 5Dforward(t) = (close_price(t+5)-close_price(t))/close_price(t)
stockforwardreturn = (stockforward - stock) / stock
#stockforwardreturn.head()
#stockforwardreturn.to_csv(os.path.join(csv_path, '5DForward.csv')) #stock
exp_return=stockforwardreturn.loc['2008-01-04':'2017-01-16']
#exp_return=exp_return.fillna(exp_return.mean())
#exp_return=exp_return.dropna(how='all',axis=1)

In [ ]:
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)


In [ ]:
def ranking(X):
    return pd.DataFrame(stats.rankdata(X,axis=0)).set_index(exp_return.index).set_axis(close.columns, axis=1,inplace=False)


In [ ]:
def rank_ic(individual):
    
    func = toolbox.compile(expr=individual)
    corr=pd.DataFrame(func(ranking(close),ranking(op),ranking(high),ranking(low),ranking(turnover),ranking(vol)).corrwith(ranking(exp_return), axis = 0)).mean()

    return corr[0],  
    

    
    
toolbox.register("evaluate", rank_ic)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)


toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))


In [ ]:
stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)

mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)

mstats.register("avg", np.mean)
mstats.register("std", np.std)
mstats.register("min", np.min)
mstats.register("max", np.max)


In [ ]:
pop = toolbox.population(n=1000)  
hof = tools.HallOfFame(50) 
pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 3, stats=mstats, halloffame=hof, verbose=True)